In [1]:
# Importing libraries
import pandas as pd
import wget
import numpy as np
import pygsheets
from IPython.display import display, HTML

In [2]:
# If you want to download the files and access them locally, you can do it this way.

# Adding URL's of the CSV Dataset
# urls = [
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv",
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",
#     "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# ]

# Downloading and Saving CSV files locally
# [wget.download(i) for i in urls]

In [3]:
# Storing the Raw CSV Dataset(s) into a DataFrame
confirmed_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")

confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46116,46274,46516,46718,46837,46837,47072,47306,47516,47716
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,37625,38182,39014,39719,40501,41302,42148,42988,43683,44436
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,82221,83199,84152,85084,85927,86730,87502,88252,88825,89416
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,6712,6745,6790,6842,6904,6955,7005,7050,7084,7127
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,15103,15139,15251,15319,15361,15493,15536,15591,15648,15729


### Now that we have had a look on the data, we can clearly define some problems and how we are going to fix them:

#### Problems:
1. Confirmed, Deaths and Recovered Cases are stored in three different datasets, which can make it difficult to visualize data
2. Dates exist as columns and give a very odd shape to the DataFrame.
3. There is also some missing data and some wrong Datatypes.

#### Fixes:
1. Merge the Confirmed, Deaths and Recovered DataFrames into a Single DataFrame.
2. Extracting the common date list from the dataset and Transpose the dates into values using the melt() function. 
3. Perform Data Cleaning for missing values and fixing wrong datatypes.
4. Add a new column namely _Active Cases_ which is calculated as **Active Cases = Confirmed - Deaths - Recovered**.
5. Aggregate Data Country/Region Wise and Group them by Date and Country/Region.
6. Add New columns **New Cases, New Deaths and New Recovered** by subtracting the Cumulative Data from the previous day.

In [4]:
# Storing the common date list in a variable.
dates = confirmed_df.columns[4:]

# Melting/Unpivoting the DataFrames from the current wide format to a long format.

confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Confirmed'
)

deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Deaths'
)

recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars= dates,
    var_name='Date',
    value_name='Recovered'
)

confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
87257,NaN,Vietnam,14.058324,108.277199,12/8/20,1377
87258,NaN,West Bank and Gaza,31.952200,35.233200,12/8/20,101109
87259,NaN,Yemen,15.552727,48.516388,12/8/20,2078
87260,NaN,Zambia,-13.133897,27.849332,12/8/20,17963


In [5]:
# Merging the DataFrames into a Single DataFrame

# Merging confirmed_df and deaths_df
final_table = confirmed_df_long.merge(
    right= deaths_df_long,
    how='left',
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
)

# Merging final_table and recovered_df
final_table = final_table.merge(
    right= recovered_df_long,
    how='left',
    on=['Province/State', 'Country/Region', 'Lat', 'Long', 'Date']
)

final_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
87257,NaN,Vietnam,14.058324,108.277199,12/8/20,1377,35,1224.0
87258,NaN,West Bank and Gaza,31.952200,35.233200,12/8/20,101109,867,75257.0
87259,NaN,Yemen,15.552727,48.516388,12/8/20,2078,606,1382.0
87260,NaN,Zambia,-13.133897,27.849332,12/8/20,17963,364,17268.0


In [6]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87262 entries, 0 to 87261
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  26726 non-null  object 
 1   Country/Region  87262 non-null  object 
 2   Lat             86940 non-null  float64
 3   Long            86940 non-null  float64
 4   Date            87262 non-null  object 
 5   Confirmed       87262 non-null  int64  
 6   Deaths          87262 non-null  int64  
 7   Recovered       80500 non-null  float64
dtypes: float64(3), int64(2), object(3)
memory usage: 6.0+ MB


### Data Cleaning
Looking at the final data, there are still some things left to do make the data fit for use
1. Replace Missing Values
2. Convert Date from String object to DateTime
3. Removing Data gathered from Cruise Ships.

We can see a lot of NaN values in the **Province/State** Column, justifiably so as a lot of counties don't report the data State/Province wise. So, we will leave that as it is for now. However, there are quite a few NaN values in the **Recovered** Column. Let's Replace those values with 0.

In [7]:
final_table['Recovered'] =  final_table['Recovered'].fillna(0)

# Converting Date from String to DateTime
final_table['Date'] = pd.to_datetime(final_table['Date'])

final_table.sort_values(by=['Country/Region', 'Date'], inplace=True)

final_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
271,NaN,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0.0
542,NaN,Afghanistan,33.939110,67.709953,2020-01-24,0,0,0.0
813,NaN,Afghanistan,33.939110,67.709953,2020-01-25,0,0,0.0
1084,NaN,Afghanistan,33.939110,67.709953,2020-01-26,0,0,0.0
...,...,...,...,...,...,...,...,...
86177,NaN,Zimbabwe,-19.015438,29.154857,2020-12-04,10547,281,8802.0
86448,NaN,Zimbabwe,-19.015438,29.154857,2020-12-05,10617,291,8844.0
86719,NaN,Zimbabwe,-19.015438,29.154857,2020-12-06,10718,291,8880.0
86990,NaN,Zimbabwe,-19.015438,29.154857,2020-12-07,10839,294,8972.0


In [8]:
# Filtering out the data from Cruise Ships
ship_rows = final_table['Province/State'].str.contains('Grand Princess') | final_table['Province/State'].str.contains('Diamond Princess') | final_table['Country/Region'].str.contains('Diamond Princess') | final_table['Country/Region'].str.contains('MS Zaandam')
full_ship = final_table[ship_rows]

# Altering the table to remove this data
final_table = final_table[~(ship_rows)]

### Data Aggregation

Now that we have our cleaned data, we will transform it such that it can give us some valuable insights and summaries.
The first thing to do is to add another column, namely **Active** Cases.

In [9]:
final_table['Active'] = final_table['Confirmed'] - final_table['Recovered'] - final_table['Deaths']

final_table

<ipython-input-9-2334a78df0a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['Active'] = final_table['Confirmed'] - final_table['Recovered'] - final_table['Deaths']


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
271,NaN,Afghanistan,33.939110,67.709953,2020-01-23,0,0,0.0,0.0
542,NaN,Afghanistan,33.939110,67.709953,2020-01-24,0,0,0.0,0.0
813,NaN,Afghanistan,33.939110,67.709953,2020-01-25,0,0,0.0,0.0
1084,NaN,Afghanistan,33.939110,67.709953,2020-01-26,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
86177,NaN,Zimbabwe,-19.015438,29.154857,2020-12-04,10547,281,8802.0,1464.0
86448,NaN,Zimbabwe,-19.015438,29.154857,2020-12-05,10617,291,8844.0,1482.0
86719,NaN,Zimbabwe,-19.015438,29.154857,2020-12-06,10718,291,8880.0,1547.0
86990,NaN,Zimbabwe,-19.015438,29.154857,2020-12-07,10839,294,8972.0,1573.0


Now, let's group our data by 'Date' and 'Country/Region'.

In [10]:
final_grouped = final_table.groupby(['Date', 'Country/Region'])[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum().reset_index()

# sum() is used here to get the total for 'Confirmed', 'Deaths', 'Recovered' and 'Active' Cases for that given Date.

final_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
60853,2020-12-08,Vietnam,1377,35,1224.0,118.0
60854,2020-12-08,West Bank and Gaza,101109,867,75257.0,24985.0
60855,2020-12-08,Yemen,2078,606,1382.0,90.0
60856,2020-12-08,Zambia,17963,364,17268.0,331.0


In [11]:
# Creating a temp DataFrame for calculating New Values
temp = final_table.groupby(['Country/Region', 'Date'])[['Confirmed', 'Deaths', 'Recovered']]
temp = temp.sum().diff().reset_index()

# diff() helps us calculate the diffrence of a cell with it's preceding cell

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

# The mask variable will help us root out the wrong values in a case where the difference is calculated between
# different countries and not between the same country for different dates

temp.loc[mask, ['Confirmed', 'Deaths', 'Recovered']] = np.nan

# Renaming Columns
temp.columns = ['Country/Region', 'Date', 'New Cases', 'New Deaths', 'New Recovered']

# Merging the DataFrames
final_grouped = pd.merge(left=final_grouped, right=temp, on=['Country/Region', 'Date'])

In [12]:
final_grouped.fillna(0, inplace=True)

# Fixing Data types
final_grouped[['Recovered', 'Active', 'New Cases', 'New Deaths', 'New Recovered']] = final_grouped[['Recovered', 'Active', 'New Cases', 'New Deaths', 'New Recovered']].astype('int')

# Fixing negative Values in the data
final_grouped['New Cases'] = final_grouped['New Cases'].apply(lambda x: 0 if x < 0 else x)

final_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New Cases,New Deaths,New Recovered
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0
1,2020-01-22,Albania,0,0,0,0,0,0,0
2,2020-01-22,Algeria,0,0,0,0,0,0,0
3,2020-01-22,Andorra,0,0,0,0,0,0,0
4,2020-01-22,Angola,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
60853,2020-12-08,Vietnam,1377,35,1224,118,10,0,0
60854,2020-12-08,West Bank and Gaza,101109,867,75257,24985,1351,19,1204
60855,2020-12-08,Yemen,2078,606,1382,90,0,-43,-172
60856,2020-12-08,Zambia,17963,364,17268,331,32,0,57


In [13]:
client = pygsheets.authorize(service_account_file=r'D:\Python\Notebooks\Covid 19 Dashboard\Covid 19 Dashboard-7fcdf6851df8.json')
sheet = client.open('Covid 19 Data with Auto Refresh')

wks = sheet[0]

# wks.rows = final_grouped[0]
wks.set_dataframe(final_grouped, start=(1,1), fit=True)

### Here is the final version of the interactive Covid 19 Dashboard that was made in tableau.

*Please open the dashboard in full screen mode to get a better viewing experience*

In [16]:
%%HTML
<div class='tableauPlaceholder' id='viz1607519024891' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard?:language=en&amp;:embed=y&amp;:display_count=y' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19Dashboard_15944506467040&#47;FinalDashboard&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1607519024891');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>